In [1]:
import sqlite3
import pandas as pd
import urllib.request

In [2]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [3]:
def execute_sql_from_file(url, cur):
    # Open and read the file as a single buffer
    sql_string = urllib.request.urlopen(url).read().decode()
    sql_commands = sql_string.split(';')
    # Execute every command from the input file
    for command in sql_commands:        
        try:
            cur.execute(command)
        except sqlite3.Error as e:
            print(e)

## Create DB in RAM from dumpfile

In [4]:
dump_url = "https://raw.githubusercontent.com/woldemarg/lightit_sql_pivot/master/db/work_log_sqlite.sql"
execute_sql_from_file(dump_url, cur)

## Check everything is OK

In [5]:
pd.read_sql_query(
    """ SELECT name
        FROM sqlite_master 
        WHERE type ='table'
        AND name NOT LIKE 'sqlite_%';
    """, con)

,name
0,public.vacancy
1,public.worktype
2,public.project
3,public.dailyplan
4,public.dailylog
5,public.worker
6,public.projectworker


## Coretable to deal with dates (UNION of 'dailylog' and 'dailyplan')  

In [6]:
core_query  = urllib.request.urlopen("https://raw.githubusercontent.com/woldemarg/lightit_sql_pivot/master/queries/query_core.sql").read().decode()
core_table = pd.read_sql_query(core_query, con)
core_table.head()

,date,time,project_id,worker_id,type
0,2020-01-30,02:00:00,1,9,spent
1,2020-01-30,04:00:00,1,9,estimated
2,2020-01-30,04:00:00,3,10,estimated
3,2020-01-30,04:00:00,3,10,spent
4,2020-01-30,08:00:00,1,5,estimated


## Select week period for:
#### start of period = 2020-02-17
#### end of period = 2020-02-21
#### "now" = 2020-02-20

In [7]:
main_query  = urllib.request.urlopen("https://raw.githubusercontent.com/woldemarg/lightit_sql_pivot/master/queries/query_main.sql").read().decode()
main_table = pd.read_sql_query(main_query, con)
main_table

,pm_name,date,dow,hours,project_id,project_name,worker_id,worker_name,vacancy_name,type
0,Артемьев Филипп Петрович,2020-02-17,Пн,7,4,Axion Project,5,Лаврентьев Павел Андреевич,Developer,spent
1,Артемьев Филипп Петрович,2020-02-17,Пн,4,3,Gemstone Project,10,Шкраба Тимофей Васильевич,QA,spent
2,Блинов Марк Григорьевич,2020-02-17,Пн,2,1,Project Galaxy,9,Некрасов Чарльз Алексеевич,QA,spent
3,Блинов Марк Григорьевич,2020-02-18,Вт,2,1,Project Galaxy,9,Некрасов Чарльз Алексеевич,QA,spent
4,Блинов Марк Григорьевич,2020-02-18,Вт,6,1,Project Galaxy,8,Шарапов Донат Вадимович,Developer,spent
5,Артемьев Филипп Петрович,2020-02-19,Ср,2,3,Gemstone Project,10,Шкраба Тимофей Васильевич,QA,spent
6,Блинов Марк Григорьевич,2020-02-19,Ср,2,1,Project Galaxy,9,Некрасов Чарльз Алексеевич,QA,spent
7,Артемьев Филипп Петрович,2020-02-20,Чт,8,4,Axion Project,5,Лаврентьев Павел Андреевич,Developer,estimated
8,Артемьев Филипп Петрович,2020-02-20,Чт,8,2,Eco Project,7,Фролов Гавриил Валерьевич,Developer,estimated
9,Артемьев Филипп Петрович,2020-02-20,Чт,4,3,Gemstone Project,10,Шкраба Тимофей Васильевич,QA,estimated


## Pivot table (report for 2020-02-17 / 2020-02-21)

In [8]:
pivot_query  = urllib.request.urlopen("https://raw.githubusercontent.com/woldemarg/lightit_sql_pivot/master/queries/query_pivot.sql").read().decode()
pivot_table = pd.read_sql_query(pivot_query, con)
pivot_table

,PM,Проект,ФИО,Пн,Вт,Ср,Чт,Пт
0,Артемьев Филипп Петрович,Axion Project,Лаврентьев Павел Андреевич,7,,,8,8
1,Артемьев Филипп Петрович,Eco Project,Фролов Гавриил Валерьевич,,,,8,8
2,Артемьев Филипп Петрович,Gemstone Project,Шкраба Тимофей Васильевич,4,,,4,4
3,Блинов Марк Григорьевич,Project Galaxy,Некрасов Чарльз Алексеевич,2,2,,4,4
4,Блинов Марк Григорьевич,Project Galaxy,Шарапов Донат Вадимович,,6,,8,8


## Another report for next week ...
#### ... as a plan (now is the beginning of reported week)

In [9]:
next_start = pivot_query.replace("2020-02-17", "2020-02-24")
next_end = next_start.replace("2020-02-21", "2020-02-28")
next_now = next_end.replace("2020-02-20", "2020-02-24")

pd.read_sql_query(next_now, con)

,PM,Проект,ФИО,Пн,Вт,Ср,Чт,Пт
0,Артемьев Филипп Петрович,Axion Project,Лаврентьев Павел Андреевич,8,8,,8,8
1,Артемьев Филипп Петрович,Eco Project,Фролов Гавриил Валерьевич,8,8,,8,8
2,Артемьев Филипп Петрович,Gemstone Project,Шкраба Тимофей Васильевич,4,4,,,
3,Блинов Марк Григорьевич,Project Galaxy,Некрасов Чарльз Алексеевич,4,4,,,
4,Блинов Марк Григорьевич,Project Galaxy,Шарапов Донат Вадимович,8,8,,8,8


#### ... as a fact (now is the end of reported week (Friday))

In [10]:
next_now = next_end.replace("2020-02-20", "2020-02-28")

pd.read_sql_query(next_now, con)

,PM,Проект,ФИО,Пн,Вт,Ср,Чт,Пт
0,Артемьев Филипп Петрович,Axion Project,Лаврентьев Павел Андреевич,,,,,8
1,Артемьев Филипп Петрович,Eco Project,Фролов Гавриил Валерьевич,,7,,,8
2,Артемьев Филипп Петрович,Gemstone Project,Шкраба Тимофей Васильевич,4,4,,,
3,Блинов Марк Григорьевич,Project Galaxy,Некрасов Чарльз Алексеевич,,,,,
4,Блинов Марк Григорьевич,Project Galaxy,Шарапов Донат Вадимович,,,,8,8
